In [1]:
from flask import *
from flask_wtf import *
from wtforms import *
from wtforms.validators import DataRequired as DR
from flask_sqlalchemy import *

In [2]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'sk'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///library.db'
db = SQLAlchemy(app)
# Gets created if it doesn't exist 
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
def restart():
    global app
    global db
    app = Flask(__name__)
    app.config['SECRET_KEY'] = 'sk'
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///library.db'
    app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
    db = SQLAlchemy(app)

In [7]:
restart()

# Create Forms using WTForms

In [9]:
# Add Author
class AuthForm(FlaskForm):
    name = StringField('Name of the Author', validators = [DR()])

# Add Book
class BookForm(FlaskForm):
    title = StringField('Title of the Book', validators = [DR()])
    author_id = IntegerField('Author ID', validators = [DR()])



# Library Database
## Schema
### Author (aid, name, books)
### Book (bid, title, author_id (Referencing to aid of Author))


In [8]:
# Author Table
class author(db.Model):
    aid = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String, nullable=False)
    books = db.relationship('Book', backref='author', lazy=True)

# Book Table
class book(db.Model):
    bid = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String, nullable=False)
    author_id = db.Column(db.Integer, db.ForeignKey('author.aid'), 
                          nullable=False)

with app.app_context(): 
    db.create_all()

In [ ]:
# View function for adding author
@app.route('/addauth', methods=['GET', 'POST'])
def addauth():
    form = AuthForm()
    if form.validate_on_submit():
        auth_name = form.name.data
        new_row = author(name=auth_name)
        db.session.add(new_row)
        db.session.commit()
    return redirect('/')

# View function for adding book
@app.route('/addbook', method=['GET', 'POST'])
def addbook():
    form = BookForm()
    if form.validate_on_submit():
        title = form.title.data
        author_id = form.author_id.data
        new_row = book(title=title, author_id=author_id)
        db.session.add(new_row)
        db.session.commit()
    return redirect('/')

# View all details on Index
@app.route('/')
def index():
    authors = author.query.all()
    books = book.query.all()
    return render_template('index.html', authors=authors, books=books)